# Импортирование библиотек

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import torchvision.models as models
import matplotlib.pyplot as plt

# Подготовка данных

In [ ]:
# Подготовка данных
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_set = CIFAR10(root='./data', train=True, download=False, transform=transform)
train_loader = DataLoader(train_set, batch_size=128, shuffle=True)

test_set = CIFAR10(root='./data', train=False, download=False, transform=transform)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False)


# Создание нейронной сети

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Инициализация модели, функции потерь и оптимизатора

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Обучение модели

In [ ]:
train_loss_history = []
test_accuracy_history = []

for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_loss_history.append(running_loss / len(train_loader))

 # Оценка модели на тестовом наборе данных

In [ ]:
correct = 0
total = 0
with torch.no_grad():
		for data in test_loader:
				images, labels = data[0].to(device), data[1].to(device)
				outputs = net(images)
				_, predicted = torch.max(outputs.data, 1)
				total += labels.size(0)
				correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
test_accuracy_history.append(test_accuracy)

print(f"Epoch {epoch + 1}, Loss: {train_loss_history[-1]}, Test Accuracy: {test_accuracy:.2f}%")

print('Finished Training')

# Вывод графика функции потерь на обучающем наборе данных

In [ ]:
plt.plot(range(1, 11), train_loss_history)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

# Вывод графика точности на тестовом наборе данных


In [ ]:
plt.plot(range(1, 11), test_accuracy_history)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Test Accuracy')
plt.show()